# 2D Image to 3D Model

## Loading Material Properties From json

In [32]:
import json
from PIL import Image
from PIL import ImageOps
import numpy as np
import trimesh
import open3d as o3d
import tensorflow as tf

with open('resized_img_processed_model_mapping.json', 'r') as f:
    img_to_mod_map = json.load(f)

with open('material_properties.json', 'r') as f:
    material_properties = json.load(f)


def load_preprocess_img(img_path):
    
    img = Image.open(img_path)

    if img.mode != 'RGB':
        print(f"Converting grayscale to RGB for: {img_path}")
        img = ImageOps.grayscale(img)
        img = ImageOps.colorize(img, black="black", white="white")

    img_array = np.array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

def simplify_mesh(mesh, target_vertices=1024):

    open3d_mesh = o3d.geometry.TriangleMesh(
        vertices=o3d.utility.Vector3dVector(mesh.vertices),
        triangles=o3d.utility.Vector3iVector(mesh.faces)
    )

    simplified_mesh = open3d_mesh.simplify_quadric_decimation(target_vertices)
    simplified_trimesh = trimesh.Trimesh(
        vertices=np.asarray(simplified_mesh.vertices),
        faces=np.asarray(simplified_mesh.triangles)
    )

    return simplified_trimesh

def upsample_mesh(mesh, target_vertices=1024):
    sampled_points, _ = trimesh.sample.sample_surface_even(mesh, target_vertices)
    return sampled_points


def load_3d_model(model_path, target_vertices=1024):
    mesh = trimesh.load(model_path)

    if len(mesh.vertices) > target_vertices:
        simplified_mesh = simplify_mesh(mesh, target_vertices)
        return simplified_mesh

    elif len(mesh.vertices) < target_vertices:
        upsampled_mesh = upsample_mesh(mesh, target_vertices)
        return upsampled_mesh
    
    return mesh


def get_material_prop(img_path, img_to_mod_map, material_properties):
    print(img_path)
    model_path = img_to_mod_map.get(img_path, None)

    if model_path is None:
        raise ValueError(f"No model found for img: {img_path}")

    material_path = model_path.replace('simple_normal_model.obj', 'model.mtl')
    material_path = material_path.replace('../model/', '')
    materials = material_properties.get(material_path, None)
    if materials is None:
        raise ValueError(f"No materials found for model: {material_path}")
    return materials


def normalize_materials(material):
    max_shine = 1000

    normalized_material = {
        'Kd': material.get('diffuse', [1.0, 1.0, 1.0]),
        'Ks': material.get('specular', [0.0, 0.0, 0.0]),
        'Ns': material.get('shininess', 96.078431) / max_shine,
        'd': material.get('transparency', 1.0),
    }

    # Flatten the normalized material into a list for easier processing
    flattened_material = normalized_material['Kd'] + normalized_material['Ks'] + [normalized_material['Ns'], normalized_material['d']]

    return flattened_material


def preprocess_image_with_material(img_path, img_to_mod_map, material_properties):
    img = load_preprocess_img(img_path)

    model_path = img_to_mod_map.get(img_path)
    mesh = load_3d_model(model_path, target_vertices=1024)

    materials = get_material_prop(img_path, img_to_mod_map, material_properties)
    normalized_materials = normalize_materials(materials)

    return img, mesh, normalized_materials

## Data Generator

In [33]:
from tensorflow.keras.utils import Sequence
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

class DataGenerator(Sequence):
    def __init__(self, img_paths, img_to_mod_map, material_properties, batch_size=32, dim=(256, 256, 3), augment=False):
        self.img_paths = img_paths
        self.img_to_mod_map = img_to_mod_map
        self.material_properties = material_properties
        self.batch_size = batch_size
        self.dim = dim
        self.augment = augment

        # Image data augmentation
        self.image_datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.1,
            height_shift_range=0.1,
            zoom_range=0.1,
            brightness_range=[0.8, 1.2],
            fill_mode='nearest'
        )

    def __len__(self):
        return int(np.floor(len(self.img_paths) / self.batch_size))

    def pad_or_trunc_mesh(self, mesh, target_vertices=1024):
        vertices = np.array(mesh)

        if vertices.shape[0] > target_vertices:
            return vertices[:target_vertices, :]
        elif vertices.shape[0] < target_vertices:
            padding = np.zeros((target_vertices - vertices.shape[0], vertices.shape[1]))
            return np.vstack([vertices, padding])
        else:
            return vertices

    def __getitem__(self, index):
        batch_img_paths = self.img_paths[index * self.batch_size:(index + 1) * self.batch_size]
        imgs = []
        materials = []
        meshes = []

        for img_path in batch_img_paths:
            img = load_preprocess_img(img_path)

            # Only squeeze if the image has 4 dimensions
            print(img.shape)
            if len(img.shape) == 4 and img.shape[0] == 1:
                img = np.squeeze(img, axis=0)  # Remove batch dimension if present

            if len(img.shape) == 2: 
                print(f"Converting grayscale to RGB for: {img_path}")
                img = np.stack([img] * 3, axis=-1) 

            if self.augment:
                print(img.shape)
                img = self.image_datagen.random_transform(img)
            
            img = np.expand_dims(img, axis=0)
            imgs.append(img)

            material = get_material_prop(img_path, self.img_to_mod_map, self.material_properties)
            normalized_material = normalize_materials(material)
            materials.append(normalized_material)

            model_path = self.img_to_mod_map.get(img_path)
            mesh = load_3d_model(model_path)
            print(f"Mesh type: {type(mesh)} for model: {model_path}")

            if hasattr(mesh, 'vertices'):
                padded_mesh = self.pad_or_trunc_mesh(mesh.vertices)
            elif isinstance(mesh, np.ndarray) or isinstance(mesh, trimesh.caching.TrackedArray):
                padded_mesh = self.pad_or_trunc_mesh(mesh)
            else:
                print(f"Warning: No vertices found in model: {model_path}. Skipping.")
                padded_mesh = np.zeros((1024, 3))

            meshes.append(padded_mesh)

        imgs = np.vstack(imgs)
        materials = np.array(materials, dtype=np.float32)
        meshes = np.array(meshes, dtype=np.float32)

        imgs_tensor = tf.convert_to_tensor(imgs, dtype=tf.float32)
        materials_tensor = tf.convert_to_tensor(materials, dtype=tf.float32)
        meshes_tensor = tf.convert_to_tensor(meshes, dtype=tf.float32)

        return (imgs_tensor, materials_tensor), meshes_tensor

    def on_epoch_end(self):
        np.random.shuffle(self.img_paths)
        print("Shuffled")


## Model Architecture

In [34]:
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50V2

image_input = Input(shape=(256, 256, 3), name='image_input')
resnet_base = ResNet50V2(weights='imagenet', include_top=False, input_tensor=image_input)

x = resnet_base.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)

material_input = Input(shape=(4,), name='material_input')
material_dense = Dense(64, activation='relu')(material_input)
material_dense = Dense(64, activation='relu')(material_dense)

combined = Concatenate()([x, material_dense])

z = Dense(256, activation='relu')(combined)
z = Dense(512, activation='relu')(z)

num_vertices = 1024
output = Dense(num_vertices * 3, activation='linear', name='output')(z)

model = Model(inputs=[image_input, material_input], outputs=output)


# Training

In [35]:
from sklearn.model_selection import train_test_split
import os

img_dir = "../resized_images/"
img_paths = []

# Iterate over the images in the directory, but only add those present in the JSON mapping
for root, dirs, files in os.walk(img_dir):
    for file in files:
        if file.endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(root, file)

            # Check if the image path is in the mapping
            if img_path in img_to_mod_map:
                img_paths.append(img_path)

print(f"Found {len(img_paths)} images.")

train_img_paths, temp_img_paths = train_test_split(img_paths, test_size=0.3, random_state=42)
val_img_paths, test_img_paths = train_test_split(temp_img_paths, test_size=0.5, random_state=42)

train_data_gen = DataGenerator(train_img_paths, img_to_mod_map, material_properties, batch_size=32, augment=True)
val_data_gen = DataGenerator(val_img_paths, img_to_mod_map, material_properties, batch_size=32, augment=False)
test_data_gen = DataGenerator(test_img_paths, img_to_mod_map, material_properties, batch_size=32, augment=False)

def generator_to_tf_dataset(generator):
    output_signature = (
        (
            tf.TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None, 8), dtype=tf.float32)
        ),
        tf.TensorSpec(shape=(None, 1024, 3), dtype=tf.float32)
    )
    return tf.data.Dataset.from_generator(lambda: generator, output_signature=output_signature)

train_dataset = generator_to_tf_dataset(train_data_gen)
val_dataset = generator_to_tf_dataset(val_data_gen)
test_dataset = generator_to_tf_dataset(test_data_gen)

model.compile(optimizer='adam', loss='mean_squared_error')
# for i in range(len(train_data_gen)):
#     print(f"Iteration: {i}")
#     data = train_data_gen[i]
#     print(f"Batch {i} processed")

history = model.fit(
    train_data_gen,
    epochs=20,
    validation_data=val_data_gen
)

Found 8521 images.
(1, 256, 256, 3)
(256, 256, 3)
../resized_images/bed/0229.jpg


only got 753/1024 samples!
only got 676/1024 samples!


Mesh type: <class 'trimesh.caching.TrackedArray'> for model: ../model/bed/IKEA_FJELLSE_1/model_simple_normal.obj
(1, 256, 256, 3)
(256, 256, 3)
../resized_images/chair/1806.jpg
Mesh type: <class 'trimesh.caching.TrackedArray'> for model: ../model/chair/IKEA_EKTORP_1/model_simple_normal.obj
(1, 256, 256, 3)
(256, 256, 3)
../resized_images/chair/2528.jpg
Mesh type: <class 'trimesh.caching.TrackedArray'> for model: ../model/chair/IKEA_MARIUS/model_simple_normal.obj
(1, 256, 256, 3)
(256, 256, 3)
../resized_images/bookcase/0151.jpg
Mesh type: <class 'trimesh.caching.TrackedArray'> for model: ../model/bookcase/IKEA_EXPEDIT_3/model_simple_normal.obj
(1, 256, 256, 3)
(256, 256, 3)


only got 845/1024 samples!


../resized_images/sofa/0340.jpg
Mesh type: <class 'trimesh.caching.TrackedArray'> for model: ../model/sofa/IKEA_EKTORP_2/model_simple_normal.obj
(1, 256, 256, 3)
(256, 256, 3)
../resized_images/chair/3150.jpg
Mesh type: <class 'trimesh.caching.TrackedArray'> for model: ../model/chair/IKEA_REIDAR/model_simple_normal.obj
(1, 256, 256, 3)
(256, 256, 3)
../resized_images/chair/2152.jpg
Mesh type: <class 'trimesh.caching.TrackedArray'> for model: ../model/chair/IKEA_JOKKMOKK/model_simple_normal.obj
(1, 256, 256, 3)
(256, 256, 3)
../resized_images/bookcase/0269.jpg
Mesh type: <class 'trimesh.caching.TrackedArray'> for model: ../model/bookcase/IKEA_LACK_3/model_simple_normal.obj
(1, 256, 256, 3)
(256, 256, 3)
../resized_images/chair/1752.jpg
Mesh type: <class 'trimesh.caching.TrackedArray'> for model: ../model/chair/IKEA_BORJE/model_simple_normal.obj
Converting grayscale to RGB for: ../resized_images/table/0847.jpg
(1, 256, 256, 3)
(256, 256, 3)
../resized_images/table/0847.jpg
Mesh type: <cl

...

only got 884/1024 samples!
only got 810/1024 samples!
only got 881/1024 samples!


../resized_images/chair/1580.jpg
Mesh type: <class 'trimesh.caching.TrackedArray'> for model: ../model/chair/IKEA_BERNHARD/model_simple_normal.obj
(1, 256, 256, 3)
(256, 256, 3)
../resized_images/chair/3572.jpg
Mesh type: <class 'trimesh.caching.TrackedArray'> for model: ../model/chair/IKEA_TOBIAS/model_simple_normal.obj
(1, 256, 256, 3)
(256, 256, 3)
../resized_images/chair/3506.jpg
Mesh type: <class 'trimesh.caching.TrackedArray'> for model: ../model/chair/IKEA_TOBIAS/model_simple_normal.obj
(1, 256, 256, 3)
(256, 256, 3)
../resized_images/desk/0085.jpg
Mesh type: <class 'trimesh.caching.TrackedArray'> for model: ../model/desk/IKEA_BESTA_2/model_simple_normal.obj
(1, 256, 256, 3)
(256, 256, 3)
../resized_images/chair/1924.jpg
Mesh type: <class 'trimesh.caching.TrackedArray'> for model: ../model/chair/IKEA_INGOLF/model_simple_normal.obj
(1, 256, 256, 3)
(256, 256, 3)
../resized_images/desk/0693.jpg
Mesh type: <class 'trimesh.caching.TrackedArray'> for model: ../model/desk/IKEA_GALANT_

only got 711/1024 samples!
only got 840/1024 samples!


Mesh type: <class 'trimesh.base.Trimesh'> for model: ../model/chair/IKEA_SNILLE_1/model_0.638510192124_0.0237072979295_simple_normal.obj
(1, 256, 256, 3)
(256, 256, 3)
../resized_images/chair/1906.jpeg
Mesh type: <class 'trimesh.caching.TrackedArray'> for model: ../model/chair/IKEA_HERMAN/model_simple_normal.obj
(1, 256, 256, 3)
(256, 256, 3)
../resized_images/bookcase/0047.jpg
Mesh type: <class 'trimesh.caching.TrackedArray'> for model: ../model/bookcase/IKEA_EXPEDIT_1/model_simple_normal.obj
(1, 256, 256, 3)
(256, 256, 3)
../resized_images/chair/1645.jpg
Mesh type: <class 'trimesh.caching.TrackedArray'> for model: ../model/chair/IKEA_BORJE/model_simple_normal.obj
(1, 256, 256, 3)
(256, 256, 3)
../resized_images/table/1382.jpg
Mesh type: <class 'trimesh.caching.TrackedArray'> for model: ../model/table/IKEA_NORDEN_1/model_simple_normal.obj
(1, 256, 256, 3)
(256, 256, 3)
../resized_images/bed/0843.jpg
Mesh type: <class 'trimesh.caching.TrackedArray'> for model: ../model/bed/IKEA_MANDAL_

ValueError: Input 1 of layer "functional_5" is incompatible with the layer: expected shape=(None, 4), found shape=(None, 8)

## Plot Data

In [ ]:
import mayplotlib.pyplot as plt

plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()